# From Simple LP to Farmer's Field (Min-Cost) — Step by Step

This notebook introduces optimization in **increasing complexity**, starting from a tiny linear program with a no-dependency solver and building up to a **farmer's field** problem with a **BQP** (binary quadratic) objective. Finally, we show how to model the simple LP in **PuLP** for an exact solution.

**Sections**
1. **Simplest LP (No dependencies)** — tiny 2-variable LP solved by brute-force grid search.
2. **Simplest LP (Pulp)** — tiny 2-variable LP solved by Pulp.
2. **Farmer's Field (Minimize Fertilizer Cost, No dependencies)** — linear penalties, randomized hill-climbing.
3. **Farmer's Field as BQP (No dependencies)** — add adjacency cost and solve with binary local search.
4. **PuLP exact model for the simple LP** — MILP formulation and solve with CBC.


## 1) Simplest LP — Grid Search (No Dependencies)
## Product mix problem - Farmers Fields

Problem: How much of each brand to purchase to minimize total cost of fertilizer given following data?

Product resource requirements and unit profit:

Two brands of fertilizer available – Super-gro, Crop-quick.

Field requires at least 16 pounds of nitrogen and 24 pounds of phosphate.

Super-gro costs: `$6 per bag`

Crop-quick: `$3 per bag`


Decision Variables:

$x_{1}$ = number of bags of Super-gro

$x_{2}$ = number of bags of Crop-quick


Cost (Z) minimization

Z = 6$x_{1}$ + 3$x_{2}$


Nitrogen Constraint

2$x_{1}$ + 4$x_{2}$ >= 16

Phosphate Constraint Check

4$x_{1}$ + 3$x_{2}$ >= 24

Non-negativitiy Constraint

$x_{1}$ > 0

$x_{2}$ > 0


In [29]:
# Define the feasibility (constraint) check
def feasible(x1, x2):
    """
    Returns True if a given (x1, x2) pair satisfies all problem constraints:
      - Nonnegativity: x1, x2 >= 0
      - Nitrogen requirement: 2x1 + 4x2 >= 16
      - Phosphate requirement: 4x1 + 3x2 >= 24
    """
    return (x1 >= 0 and x2 >= 0 and
            2*x1 + 4*x2 >= 16 and   # nitrogen constraint
            4*x1 + 3*x2 >= 24)      # phosphate constraint


# Define the objective (cost) function
def objective(x1, x2):
    """
    Computes the total cost Z = 6x1 + 3x2
    where:
      x1 = number of bags of Super-Gro ($6 per bag)
      x2 = number of bags of Crop-Quick ($3 per bag)
    """
    return 6*x1 + 3*x2


# Initialize variables to track the best (minimum cost) solution
best = None
best_val = float('inf')  # Start with infinity since we are minimizing


# Set the grid step size for brute-force search
# A smaller step (e.g., 0.5 or 0.1) increases accuracy but takes more time
step = 1.0

# Search over all combinations of x1 and x2 within a reasonable range (0–120)
for i in range(0, 121):
    x1 = i * step
    for j in range(0, 121):
        x2 = j * step
        
        # Skip infeasible solutions that don't satisfy constraints
        if not feasible(x1, x2):
            continue
        
        # Compute the objective value for feasible (x1, x2)
        val = objective(x1, x2)
        
        # Keep the solution if it yields a lower cost than the best so far
        if val < best_val:          # Minimization condition
            best_val = val
            best = (x1, x2)

# Print the best solution found on the grid
print('Best (grid): x1, x2 =', best, ' objective =', best_val)

Best (grid): x1, x2 = (0.0, 8.0)  objective = 24.0


## 🧮 Introduction to PuLP for Linear Programming

[PuLP](https://coin-or.github.io/pulp/) is a powerful Python library for formulating and solving **linear programming (LP)** and **mixed-integer programming (MIP)** problems.  
It allows you to express optimization problems in Python using familiar algebraic notation—defining **decision variables**, **objective functions**, and **constraints**—and then automatically calls a solver (like CBC, which comes built-in).

### 💡 What You Can Do with PuLP
- Model problems such as minimizing costs or maximizing profits  
- Add constraints representing resource limits or requirements  
- Solve linear, integer, and mixed-integer optimization problems  
- Retrieve optimal variable values and objective function results

### ⚙️ Installation
PuLP is not always pre-installed in some notebook environments (like Google Colab).  
If you get an error saying `No module named 'pulp'`, install it by running:

```python
!pip install pulp

In [30]:
# Import the PuLP library (Python library for Linear Programming)
import pulp as pl

# ------------------------------------------------------------------
# 1. Create an optimization problem instance
# ------------------------------------------------------------------
# "MIPModel" is the name of the model, and we specify that it is a
# minimization problem using pl.LpMinimize.
opt_model = pl.LpProblem("MIPModel", pl.LpMinimize)


# ------------------------------------------------------------------
# 2. Define the Decision Variables
# ------------------------------------------------------------------
# Each variable represents the number of fertilizer bags to use.
# - x1: number of Super-Gro bags
# - x2: number of Crop-Quick bags
# The lowBound=0 enforces the non-negativity constraint (x1, x2 ≥ 0).
# cat=pl.LpInteger restricts the variables to integer values
# (useful if partial bags are not allowed).
x1 = pl.LpVariable(cat=pl.LpInteger, lowBound=0, name="$x_{1}$")
x2 = pl.LpVariable(cat=pl.LpInteger, lowBound=0, name="$x_{2}$")


# ------------------------------------------------------------------
# 3. Define the Objective Function
# ------------------------------------------------------------------
# Minimize total fertilizer cost:
#     Z = 6x1 + 3x2
# This line adds the objective function to the model.
opt_model += 6 * x1 + 3 * x2


# ------------------------------------------------------------------
# 4. Define the Constraints
# ------------------------------------------------------------------
# Constraint 1: Nitrogen requirement
#     2x1 + 4x2 ≥ 16
opt_model += 2 * x1 + 4 * x2 >= 16

# Constraint 2: Phosphate requirement
#     4x1 + 3x2 ≥ 24
opt_model += 4 * x1 + 3 * x2 >= 24


# ------------------------------------------------------------------
# 5. Display the model formulation
# ------------------------------------------------------------------
# Printing the model shows the full LP formulation, including the
# objective function and constraints.
opt_model

MIPModel:
MINIMIZE
6*$x_{1}$ + 3*$x_{2}$ + 0
SUBJECT TO
_C1: 2 $x_{1}$ + 4 $x_{2}$ >= 16

_C2: 4 $x_{1}$ + 3 $x_{2}$ >= 24

VARIABLES
0 <= $x_{1}$ Integer
0 <= $x_{2}$ Integer

In [31]:
# ------------------------------------------------------------------
# 6. Solve the model and print results
# ------------------------------------------------------------------

# Choose a solver (CBC ships with PuLP). Set msg=True to see solver logs.
result_status = opt_model.solve(pl.PULP_CBC_CMD(msg=False))

# Human-readable status
status_str = pl.LpStatus.get(opt_model.status, "Unknown")
print(f"Solve status: {status_str}")

if status_str not in ("Optimal", "Feasible"):
    print("Model did not return an optimal/feasible solution.")
else:
    # Decision variable values
    x1_val = pl.value(x1)
    x2_val = pl.value(x2)
    obj_val = pl.value(opt_model.objective)

    print("\nOptimal solution:")
    print(f"  x1 (Super-Gro bags) : {x1_val}")
    print(f"  x2 (Crop-Quick bags): {x2_val}")
    print(f"  Minimum total cost  : {obj_val}")

    # Optional: report constraint activity and slack
    # (slack >= 0 means the constraint is satisfied with that margin)
    print("\nConstraint check:")
    for name, constr in opt_model.constraints.items():
        lhs_val = constr.value() + (constr.constant if hasattr(constr, "constant") else 0.0)
        # In PuLP, constr.slack is available after solving (CBC).
        try:
            slack = constr.slack
        except Exception:
            slack = None
        print(f"  {name}: LHS={lhs_val:.3f}, sense='{constr.sense}', RHS={constr.constant}, slack={slack}")

Solve status: Optimal

Optimal solution:
  x1 (Super-Gro bags) : 0.0
  x2 (Crop-Quick bags): 8.0
  Minimum total cost  : 24.0

Constraint check:
  _C1: LHS=0.000, sense='1', RHS=-16, slack=-16.0
  _C2: LHS=-24.000, sense='1', RHS=-24, slack=-0.0


## 🌾 Extension: Selecting Fields to Plant — A Binary Linear Programming Problem

In the previous example, we decided **how many bags** of two fertilizers to buy to meet nutrient requirements at the lowest cost.  
Now, we extend this idea to a more complex and realistic **decision problem** faced by farmers:

> Instead of deciding *how much fertilizer to buy*, we decide **which fields to plant**.

Each field differs in:
- **Cost** (e.g., fertilizer and preparation costs if planted)  
- **Acreage** (the size of the field)  
- **Water usage** (how much irrigation it requires)

The farmer wants to:
- **Plant enough land** to reach a required total acreage (e.g., 25 acres)  
- **Stay within available water capacity** (e.g., 50 water units)  
- **Minimize the total planting cost**

### 🔢 Decision Variables
For each field \( i \):
\[
x_i = 
\begin{cases}
1 & \text{if field } i \text{ is planted} \\
0 & \text{if field } i \text{ is left fallow}
\end{cases}
\]

### 🎯 Objective Function
Minimize total planting cost:
\[
\text{Minimize } Z = \sum_i (\text{Cost}_i \times x_i)
\]

### 📏 Constraints
\[
\begin{aligned}
\sum_i (\text{Acreage}_i \times x_i) &\geq \text{AREA\_REQ} &\text{(meet total acreage requirement)} \\
\sum_i (\text{Water}_i \times x_i) &\leq \text{WATER\_CAP} &\text{(stay within water limit)} \\
x_i &\in \{0,1\} &\text{(binary planting decision)}
\end{aligned}
\]

### 🧠 Solving the Problem
We’ll start with a **dependency-free heuristic** that searches for good solutions using random local improvements and penalty functions for constraint violations.  
This mimics how optimization works conceptually—exploring and refining candidate solutions without relying on a solver.

Then, we’ll revisit the same problem using **PuLP**, a Python linear programming library, to find the **exact optimal solution**.

This transition demonstrates how the same logic—**objective, constraints, and decision variables**—can be scaled from simple linear programs to **mixed-integer programs (MIPs)** that include yes/no (binary) decisions.


In [32]:
from typing import List, Tuple
import random

# -----------------------------
# Problem data (edit as needed)
# -----------------------------
# Per-field parameters: choose field i (1) or skip (0)
Cost    = [9, 7, 6, 8, 12, 5, 11, 4, 4, 10, 3, 6]   # fertilizer cost if planted
Acreage = [5, 6, 3, 4,  8, 3,  7, 2, 2,  6, 1, 4]   # acres contributed if planted
Water   = [9, 8, 5, 7, 12, 4, 10, 3, 2,  8, 2, 6]   # water units consumed if planted

AREA_REQ  = 25   # minimum total acres that must be planted
WATER_CAP = 50   # maximum total water available

n = len(Cost)
assert len(Acreage)==n and len(Water)==n, "Data length mismatch"

# -----------------------------
# Penalty weights
# -----------------------------
# Large penalties push the search toward feasibility.
PEN_REQ    = 1e3   # penalty per missing acre (area shortfall)
PEN_CAP    = 1e3   # penalty per extra unit of water (cap violation)
PEN_BINARY = 1.0   # smooth binary encouragement: x*(1-x) is 0 at 0/1 and >0 in-between

def obj_with_penalties(xx: List[float]) -> float:
    """
    Smooth objective for heuristic search:
      cost + penalties for constraint violations + 'binary' encouragement.
    xx are relaxed in [0,1]; later we interpret >= 0.5 as 'select'.
    """
    total_cost  = sum(Cost[i]*xx[i]     for i in range(n))
    total_area  = sum(Acreage[i]*xx[i]  for i in range(n))
    total_water = sum(Water[i]*xx[i]    for i in range(n))

    # constraint violation penalties (0 if feasible)
    viol = 0.0
    if total_area < AREA_REQ:
        viol += PEN_REQ * (AREA_REQ - total_area)
    if total_water > WATER_CAP:
        viol += PEN_CAP * (total_water - WATER_CAP)

    # encourage xx to be close to 0 or 1 (relaxed -> binary)
    bin_pen = PEN_BINARY * sum(x*(1.0-x) for x in xx)

    return total_cost + viol + bin_pen

def solve_dependency_free(iters=30000, seed=42, step=0.25) -> Tuple[List[float], float]:
    """
    Simple randomized local search in [0,1]^n:
      - Start random
      - Nudge one coordinate at a time
      - Occasionally restart
    Returns best relaxed vector and its penalized objective.
    """
    random.seed(seed)
    # start with a random relaxed solution in [0,1]
    x = [random.random() for _ in range(n)]
    f = obj_with_penalties(x)
    best_x, best_f = x[:], f

    for _ in range(iters):
        y = x[:]
        i = random.randrange(n)
        y[i] = min(1.0, max(0.0, y[i] + random.uniform(-step, step)))
        fy = obj_with_penalties(y)

        if fy < f:  # keep improvement
            x, f = y, fy
            if f < best_f:
                best_x, best_f = x[:], f
        # occasional random restart to escape local minima
        elif random.random() < 0.001:
            x = [random.random() for _ in range(n)]
            f = obj_with_penalties(x)
            if f < best_f:
                best_x, best_f = x[:], f

    return best_x, best_f

def summarize_relaxed(x_relaxed: List[float], thresh=0.5):
    """
    Convert relaxed solution to a binary plan via thresholding,
    then compute totals and report feasibility.
    """
    picks = [i for i, xi in enumerate(x_relaxed) if xi >= thresh]
    area  = sum(Acreage[i] for i in picks)
    water = sum(Water[i]   for i in picks)
    cost  = sum(Cost[i]    for i in picks)
    feasible = (area >= AREA_REQ) and (water <= WATER_CAP)
    return picks, area, water, cost, feasible

# Run the dependency-free heuristic
x_relaxed, f_pen = solve_dependency_free()
picks, area, water, cost, feas = summarize_relaxed(x_relaxed)

print('Dependency-free solution (approx):')
print('  chosen fields:', picks)
print('  acres:', area, '| water:', water, '| fertilizer cost:', cost, '| feasible:', feas)
print('  penalized objective:', round(f_pen, 3))

Dependency-free solution (approx):
  chosen fields: [1, 2, 4, 8, 9, 11]
  acres: 29 | water: 41 | fertilizer cost: 45 | feasible: True
  penalized objective: 40.912


In [33]:
# pip install pulp   # <- run this once if PuLP isn't available

import pulp as pl

# Create model: minimize total cost
m = pl.LpProblem("FieldSelection", pl.LpMinimize)

# Binary decision variables: 1 = plant field i, 0 = skip
x = [pl.LpVariable(f"x_{i}", lowBound=0, upBound=1, cat=pl.LpBinary) for i in range(n)]

# Objective
m += pl.lpSum(Cost[i] * x[i] for i in range(n))

# Constraints
m += pl.lpSum(Acreage[i] * x[i] for i in range(n)) >= AREA_REQ,  "MinArea"
m += pl.lpSum(Water[i]   * x[i] for i in range(n)) <= WATER_CAP, "WaterCap"

# Solve (CBC ships with PuLP)
m.solve()

# Extract solution
picks_exact = [i for i in range(n) if pl.value(x[i]) > 0.5]
area_exact  = sum(Acreage[i] for i in picks_exact)
water_exact = sum(Water[i]   for i in picks_exact)
cost_exact  = sum(Cost[i]    for i in picks_exact)

print("\nExact PuLP solution:")
print("  chosen fields:", picks_exact)
print("  acres:", area_exact, "| water:", water_exact, "| fertilizer cost:", cost_exact)
print("  objective (min cost):", pl.value(m.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/jasonkuruzovich/anaconda3/envs/quantum/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/bv/2rb8dc7100ncxqhclz10zf5c0000gn/T/89a54f9f66ca4613ae0733a75211625a-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/bv/2rb8dc7100ncxqhclz10zf5c0000gn/T/89a54f9f66ca4613ae0733a75211625a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 68 RHS
At line 71 BOUNDS
At line 84 ENDATA
Problem MODEL has 2 rows, 12 columns and 24 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 36 - 0.00 seconds
Cgl0004I processed model has 2 rows, 12 columns (12 integer (12 of which binary)) and 24 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 36
Cbc003

## 🌳 Understanding How the Solver Finds the Optimal Integer Solution: Branch and Bound

When we introduce **integer or binary decision variables** (e.g., \( x_i \in \{0,1\} \)), our problem becomes **non-convex** and cannot be solved directly by linear programming methods.  
PuLP handles this kind of problem by using a powerful algorithm called **Branch and Bound**.

### 🧠 The Core Idea

The Branch and Bound (B&B) algorithm systematically explores possible combinations of variable values without checking every possibility explicitly.

1. **Relax the integrality constraints**  
   - First, the solver ignores the integer restrictions (treats variables as continuous).  
   - It solves the resulting **linear relaxation** to find a lower bound on the best possible solution.

2. **Branch**  
   - If the relaxation gives non-integer values (e.g., \( x_1 = 2.7 \)), the solver creates two new subproblems:  
     - one where \( x_1 \le 2 \)  
     - one where \( x_1 \ge 3 \)  
   - Each branch represents a smaller, more restricted version of the problem.

3. **Bound**  
   - Each subproblem’s LP relaxation gives a **bound** (an estimate of the best possible solution within that branch).  
   - If a branch’s bound is worse than a known feasible integer solution, it’s **pruned** — no need to explore it further.

4. **Search and Prune**  
   - The solver continues branching and bounding recursively, cutting off subproblems that can’t lead to a better integer solution.  
   - This dramatically reduces the number of cases that need to be explicitly checked.

5. **Termination**  
   - The process continues until all branches are either solved or pruned.  
   - The best remaining integer-feasible solution is guaranteed to be optimal.

### ⚙️ Why This Matters
Branch and Bound provides a **systematic and efficient** way to solve NP-hard problems like our **integer fertilizer model** or the **field selection extension**.  
While the worst-case runtime grows exponentially with problem size, in practice it performs well on many real-world problems due to smart bounding, cutting planes, and heuristics.

---

📊 **Key takeaway:**  
Linear Programming solvers find *continuous* optima efficiently.  
Mixed Integer Programming solvers (like CBC in PuLP) use **Branch and Bound** to extend that efficiency to discrete, yes/no, and integer decision problems — the kind we often face in management, operations, and resource allocation.